<a href="https://colab.research.google.com/github/endophenotype/Spark/blob/main/Spark_%D0%A0%D0%B5%D0%B0%D0%BB%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_%D0%9A%D0%BE%D0%BC%D0%B0%D0%BD%D0%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [ ]:
import findspark
findspark.init()

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.pandas as ps
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark

**1.Объединение двух RDD**

In [ ]:
from pyspark import SparkContext, SparkConf
appName = 'appName'
master = 'local[*]'
conf = SparkConf().setAppName(appName).setMaster(master)
sc = spark.sparkContext
R = sc.textFile("/content/R.txt");
print(R.collect())
S = sc.textFile("/content/S.txt");
print(S.collect())
print(type(R))
print(type(S))
r1 = R.map(lambda s: s.split(","))
print(r1.collect())
r2 = r1.flatMap(lambda s: [(s[0], s[1])])
print(r2.collect())
s1 = S.map(lambda s: s.split(","))
print(s1.collect())
s2 = s1.flatMap(lambda s: [(s[0], s[1])])
print(s2.collect())
RjoinedS = r2.join(s2)
RjoinedS.collect()

['k1,v1', 'k1,v2', 'k2,v3', 'k2,v4', 'k3,v7', 'k3,v8', 'k3,v9']
['k1,v11', 'k1,v22', 'k1,v33', 'k2,v55', 'k4,v77', 'k5,v88']
<class 'pyspark.rdd.RDD'>
<class 'pyspark.rdd.RDD'>
[['k1', 'v1'], ['k1', 'v2'], ['k2', 'v3'], ['k2', 'v4'], ['k3', 'v7'], ['k3', 'v8'], ['k3', 'v9']]
[('k1', 'v1'), ('k1', 'v2'), ('k2', 'v3'), ('k2', 'v4'), ('k3', 'v7'), ('k3', 'v8'), ('k3', 'v9')]
[['k1', 'v11'], ['k1', 'v22'], ['k1', 'v33'], ['k2', 'v55'], ['k4', 'v77'], ['k5', 'v88']]
[('k1', 'v11'), ('k1', 'v22'), ('k1', 'v33'), ('k2', 'v55'), ('k4', 'v77'), ('k5', 'v88')]


[('k1', ('v1', 'v11')),
 ('k1', ('v1', 'v22')),
 ('k1', ('v1', 'v33')),
 ('k1', ('v2', 'v11')),
 ('k1', ('v2', 'v22')),
 ('k1', ('v2', 'v33')),
 ('k2', ('v3', 'v55')),
 ('k2', ('v4', 'v55'))]

**2. Использование map() в RDD**
Работа с эллементами массивов, сначала отображаем набор чисел, затем прибавляем к каждому значению 2, в конце показываем квадраты чисел в набор данных

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
nums = sc.parallelize([1, 2, 3, 4, 5])
print(nums.collect())
bytwo = nums.map(lambda x: x + 2)
print(bytwo.collect())
squared = nums.map(lambda x: x * x)
print(squared.collect())

[1, 2, 3, 4, 5]
[3, 4, 5, 6, 7]
[1, 4, 9, 16, 25]


**3. Использование filter() в RDD**
Отображает массив, затем все нечетные и в конце все чётные числа в массиве

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
nums = sc.parallelize([1, 2, 3, 4, 5, 6, 7])
print(nums.collect())
filtered1 = nums.filter(lambda x : x % 2 == 1)
print(filtered1.collect())
filtered2 = nums.filter(lambda x : x % 2 == 0)
print(filtered2.collect())

[1, 2, 3, 4, 5, 6, 7]
[1, 3, 5, 7]
[2, 4, 6]


**4. Среднее значение в RDD**

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
nums = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 20])
nums.collect()
sumAndCount = nums.map(lambda x: (x, 1)).fold((0, 0), (lambda x, y: (x[0] + y[0], x[1] + y[1])))
print(sumAndCount)
avg = float(sumAndCount[0]) / float(sumAndCount[1])
print(avg)

(56, 9)
6.222222222222222


**5. Сортировки в RDD и добавление индекса**

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
a = [('g1', 2), ('g2', 4), ('g3', 3), ('g4', 8)]
rdd = sc.parallelize(a);
print(rdd.collect())
sorted = rdd.sortByKey()
print(sorted.collect())
rdd2 = rdd.flatMap(lambda s: [(s[1], s[0])])
print(rdd2.collect())
sorted = rdd2.sortByKey()
print(sorted.collect())
sorted = rdd2.sortByKey(False)
print(sorted.collect())
sorted = rdd2.sortByKey()
print(sorted.collect())
indices = sorted.zipWithIndex()
print(indices.collect())

[('g1', 2), ('g2', 4), ('g3', 3), ('g4', 8)]
[('g1', 2), ('g2', 4), ('g3', 3), ('g4', 8)]
[(2, 'g1'), (4, 'g2'), (3, 'g3'), (8, 'g4')]
[(2, 'g1'), (3, 'g3'), (4, 'g2'), (8, 'g4')]
[(8, 'g4'), (4, 'g2'), (3, 'g3'), (2, 'g1')]
[(2, 'g1'), (3, 'g3'), (4, 'g2'), (8, 'g4')]
[((2, 'g1'), 0), ((3, 'g3'), 1), ((4, 'g2'), 2), ((8, 'g4'), 3)]


**6. Произведение ряда в RDD**

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
numbers = sc.parallelize([1, 2, 3, 4])
mult = numbers.fold(1, (lambda x, y: x * y))
print(mult)

24


**7. Сортировки в RDD**

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
lines = sc.textFile('/content/data.txt', 1);
print(lines.collect())
frequencies = lines.flatMap(lambda x: x.split(' ')).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
print(frequencies.collect())
print(frequencies.count())
sorted = frequencies.sortByKey()
print(sorted.collect())
sortedDescending = frequencies.sortByKey(False)
print(sortedDescending.collect())

['crazy crazy fox jumped', 'crazy fox jumped', 'fox is fast', 'fox is smart', 'dog is smart']
[('crazy', 3), ('fox', 4), ('jumped', 2), ('is', 3), ('fast', 1), ('smart', 2), ('dog', 1)]
7
[('crazy', 3), ('dog', 1), ('fast', 1), ('fox', 4), ('is', 3), ('jumped', 2), ('smart', 2)]
[('smart', 2), ('jumped', 2), ('is', 3), ('fox', 4), ('fast', 1), ('dog', 1), ('crazy', 3)]


**8. Суммирование ряда в RDD**

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
numbers = sc.parallelize([1, 2, 3, 4])
sumRDD = numbers.fold(0, (lambda x, y: x + y))
print(sumRDD)

10


**9. Объединение в RDD**

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
d1= [('k1', 1), ('k2', 2), ('k3', 5)]
d2= [('k1', 3), ('k2',4), ('k4', 8)]
rdd1 = sc.parallelize(d1)
print(rdd1.collect())
rdd2 = sc.parallelize(d2)
print(rdd2.collect())
rdd3 = rdd1.union(rdd2)
print(rdd3.collect())
rdd4 = rdd3.reduceByKey(lambda x,y: x+y)
print(rdd4.collect())

[('k1', 1), ('k2', 2), ('k3', 5)]
[('k1', 3), ('k2', 4), ('k4', 8)]
[('k1', 1), ('k2', 2), ('k3', 5), ('k1', 3), ('k2', 4), ('k4', 8)]
[('k1', 4), ('k2', 6), ('k4', 8), ('k3', 5)]


**10. Подсчет совпадений в RDD**

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
lines = sc.textFile("/content/data.txt")
print(lines.collect())
bigrams = lines.map(lambda s : s.split(" ")).flatMap(lambda s: [((s[i],s[i+1]),1) for i in range (0, len(s)-1)])
print(bigrams.collect())
counts = bigrams.reduceByKey(lambda x, y : x+y)
print(counts.collect())

['crazy crazy fox jumped', 'crazy fox jumped', 'fox is fast', 'fox is smart', 'dog is smart']
[(('crazy', 'crazy'), 1), (('crazy', 'fox'), 1), (('fox', 'jumped'), 1), (('crazy', 'fox'), 1), (('fox', 'jumped'), 1), (('fox', 'is'), 1), (('is', 'fast'), 1), (('fox', 'is'), 1), (('is', 'smart'), 1), (('dog', 'is'), 1), (('is', 'smart'), 1)]
[(('crazy', 'crazy'), 1), (('fox', 'jumped'), 2), (('is', 'smart'), 2), (('crazy', 'fox'), 2), (('fox', 'is'), 2), (('is', 'fast'), 1), (('dog', 'is'), 1)]


**11. Декартовые координаты (перебор значений) этого RDD**


In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
a = [('k1','v1'), ('k2', 'v2')]
b = [('k3','v3'), ('k4', 'v4'), ('k5', 'v5') ]
rdd1= sc.parallelize(a)
print(rdd1.collect())
rdd2= sc.parallelize(b)
print(rdd2.collect())
rdd3 = rdd1.cartesian(rdd2)
print(rdd3.collect())

[('k1', 'v1'), ('k2', 'v2')]
[('k3', 'v3'), ('k4', 'v4'), ('k5', 'v5')]
[(('k1', 'v1'), ('k3', 'v3')), (('k1', 'v1'), ('k4', 'v4')), (('k1', 'v1'), ('k5', 'v5')), (('k2', 'v2'), ('k3', 'v3')), (('k2', 'v2'), ('k4', 'v4')), (('k2', 'v2'), ('k5', 'v5'))]


**12. Комбинировать по ключу RDD**

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
input = [("k1", 1), ("k1", 2), ("k1", 3), ("k1", 4), ("k1", 5), ("k2", 6), ("k2", 7), ("k2", 8), ("k3", 10), ("k3", 12)]
rdd = sc.parallelize(input)
sumCount = rdd.combineByKey((lambda x: (x, 1)), (lambda x, y: (x[0] + y, x[1] + 1)), (lambda x, y: (x[0] + y[0], x[1] + y[1])))
print(sumCount.collect())
avg = sumCount.mapValues( lambda v : v[0] / v[1])
print(avg.collect())

[('k2', (21, 3)), ('k1', (15, 5)), ('k3', (22, 2))]
[('k2', 7.0), ('k1', 3.0), ('k3', 11.0)]


**13. Сумма каждого раздела RDD**

In [ ]:
def adder(iterator):
  yield sum(iterator)
def f(iterator):
  for x in iterator:
    print(x)
  print("===")
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
rdd = sc.parallelize(numbers, 3)
print(rdd.collect())
print(rdd.getNumPartitions())
rdd.foreachPartition(f)
print(rdd.mapPartitions(adder).collect())

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
3
[6, 15, 34]


**14. Нахождение минимума и максимума в RDD**

In [ ]:
def minmax(iterator):
  firsttime = 0
  #min = 0;
  #max = 0;
  for x in iterator:
    if (firsttime == 0):
      min = x;
      max = x;
      firsttime = 1
    else:
      if x > max:
        max = x
      if x < min:
        min = x
  return [(min, max)]
data = [10, 20, 3, 4, 5, 2, 2, 20, 20, 10]
print (minmax(data))

[(2, 20)]


In [ ]:
def minmax(iterator):
  firsttime = 0
  for x in iterator:
    if (firsttime == 0):
      min = x;
      max = x;
      firsttime = 1
    else:
      if x > max:
        max = x
      if x < min:
        min = x
  return [(min, max)]
def f(iterator):
  for x in iterator:
    print(x)
  print("===")
from pyspark import SparkContext, SparkConf
appName = 'appName'
master = 'local[*]'
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
data = [10, 20, 3, 4, 5, 2, 2, 20, 20, 10]
rdd = sc.parallelize(data, 3)
print(rdd.getNumPartitions())
print(rdd.collect())
rdd.foreachPartition(f)
minmaxlist = rdd.mapPartitions(minmax).collect()
print(minmaxlist)
print(min(minmaxlist))
print(max(minmaxlist))

3
[10, 20, 3, 4, 5, 2, 2, 20, 20, 10]
[(3, 20), (2, 5), (2, 20)]
(2, 5)
(3, 20)


**15. pyspark UDF (User Defined Functions)**

In [ ]:
from pyspark.sql import Row

In [ ]:
spark = SparkSession.builder.appName("Example").getOrCreate()
data = spark.createDataFrame(
  [Row(zip_code='94087', city='Sunnyvale'),
  Row(zip_code='94088', city='Cupertino'),
  Row(zip_code='95055', city='Santa Clara'),
  Row(zip_code='95054', city='Palo Alto')])
data.show()

+--------+-----------+
|zip_code|       city|
+--------+-----------+
|   94087|  Sunnyvale|
|   94088|  Cupertino|
|   95055|Santa Clara|
|   95054|  Palo Alto|
+--------+-----------+



In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql import types as T
@udf(T.MapType(T.StringType(), T.StringType()))
def create_structure(zip_code, city):
  return {zip_code: city}
data.withColumn('structure', create_structure(data.zip_code, data.city)).toJSON().collect()
[
'{"zip_code":"94087","city":"Sunnyvale","structure":{"94087":"Sunnyvale"}}',
'{"zip_code":"94088","city":"Cupertino","structure":{"94088":"Cupertino"}}',
'{"zip_code":"95055","city":"Santa Clara","structure":{"95055":"Santa Clara"}}',
'{"zip_code":"95054","city":"Palo Alto","structure":{"95054":"Palo Alto"}}'
]
data.withColumn('structure', create_structure(data.zip_code, data.city)).show(truncate=False)

+--------+-----------+----------------------+
|zip_code|city       |structure             |
+--------+-----------+----------------------+
|94087   |Sunnyvale  |{94087 -> Sunnyvale}  |
|94088   |Cupertino  |{94088 -> Cupertino}  |
|95055   |Santa Clara|{95055 -> Santa Clara}|
|95054   |Palo Alto  |{95054 -> Palo Alto}  |
+--------+-----------+----------------------+



**16. Использование Split функции**

In [ ]:
#Example-1: Split RDD<String> into Tokens
data = ["abc,de", "abc,de,ze", "abc,de,ze,pe"]
rdd = sc.parallelize(data)
print(rdd.collect())
print(rdd.count())
rdd2 = rdd.flatMap(lambda x : x.split(","))
print(rdd2.collect())
print(rdd2.count())
print('////////////////////////////////////////////////////////////////////////')
#Example-2: Create Key-Value Pairs
data2 = ["abc,de", "xyz,deeee,ze", "abc,de,ze,pe", "xyz,bababa"]
rdd3 = sc.parallelize(data2)
print(rdd3.collect())
rdd4 = rdd3.map(lambda x : (x.split(",")[0], x.split(",")[1]))
print(rdd4.collect())

['abc,de', 'abc,de,ze', 'abc,de,ze,pe']
3
['abc', 'de', 'abc', 'de', 'ze', 'abc', 'de', 'ze', 'pe']
9
////////////////////////////////////////////////////////////////////////
['abc,de', 'xyz,deeee,ze', 'abc,de,ze,pe', 'xyz,bababa']
[('abc', 'de'), ('xyz', 'deeee'), ('abc', 'de'), ('xyz', 'bababa')]


**17. top-N**

In [ ]:
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
nums = [10, 1, 2, 9, 3, 4, 5, 6, 7]
print(sc.parallelize(nums).takeOrdered(3))
print(sc.parallelize(nums).takeOrdered(3, key=lambda x: -x))
kv = [(10,"z1"), (1,"z2"), (2,"z3"), (9,"z4"), (3,"z5"), (4,"z6"), (5,"z7"), (6,"z8"), (7,"z9")]
print(sc.parallelize(kv).takeOrdered(3))
print(sc.parallelize(kv).takeOrdered(3, key=lambda x: -x[0]))

[1, 2, 3]
[10, 9, 7]
[(1, 'z2'), (2, 'z3'), (3, 'z5')]
[(10, 'z1'), (9, 'z4'), (7, 'z9')]


**18. Подсчет слов**

In [ ]:
from pyspark import SparkContext, SparkConf
sc.stop()
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)
lines = sc.textFile("data.txt", 1)
debuglines = lines.collect();
print(debuglines)

words = lines.flatMap(lambda x: x.split(' '))
ones = words.map(lambda x: (x, 1))
debugwords = words.collect();
print(debugwords)
counts = ones.reduceByKey(lambda x, y: x + y)
debugcounts = counts.collect();
print(debugcounts)
counts.saveAsTextFile("outputWords ")

['crazy crazy fox jumped', 'crazy fox jumped', 'fox is fast', 'fox is smart', 'dog is smart']
['crazy', 'crazy', 'fox', 'jumped', 'crazy', 'fox', 'jumped', 'fox', 'is', 'fast', 'fox', 'is', 'smart', 'dog', 'is', 'smart']
[('crazy', 3), ('fox', 4), ('jumped', 2), ('is', 3), ('fast', 1), ('smart', 2), ('dog', 1)]


**19. Чтение данных из URL**

In [ ]:
from pyspark.sql import SparkSession, functions as F
from urllib.request import urlopen
spark = SparkSession.builder.getOrCreate()
url = 'https://www.cbr-xml-daily.ru/daily_json.js'
jsonData = urlopen(url).read().decode('utf-8')
rdd = spark.sparkContext.parallelize([jsonData])
#df = spark.read.json(rdd)
df = spark.read.option("inferSchema", "true").option("multiline", "true").json(rdd)
df.printSchema()
df.select("Valute.*").show(1,truncate=90,vertical=True)
df.select("Valute.XDR.*").show(1,truncate=90,vertical=True)
df.select("Valute.XDR.*").show(1,truncate=90,vertical=False)

root
 |-- Date: string (nullable = true)
 |-- PreviousDate: string (nullable = true)
 |-- PreviousURL: string (nullable = true)
 |-- Timestamp: string (nullable = true)
 |-- Valute: struct (nullable = true)
 |    |-- AED: struct (nullable = true)
 |    |    |-- CharCode: string (nullable = true)
 |    |    |-- ID: string (nullable = true)
 |    |    |-- Name: string (nullable = true)
 |    |    |-- Nominal: long (nullable = true)
 |    |    |-- NumCode: string (nullable = true)
 |    |    |-- Previous: double (nullable = true)
 |    |    |-- Value: double (nullable = true)
 |    |-- AMD: struct (nullable = true)
 |    |    |-- CharCode: string (nullable = true)
 |    |    |-- ID: string (nullable = true)
 |    |    |-- Name: string (nullable = true)
 |    |    |-- Nominal: long (nullable = true)
 |    |    |-- NumCode: string (nullable = true)
 |    |    |-- Previous: double (nullable = true)
 |    |    |-- Value: double (nullable = true)
 |    |-- AUD: struct (nullable = true)
 |    |

20. Построение графиков всех столбцов

In [ ]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
pser = pd.Series(np.random.randn(1000),
 index=pd.date_range('1/1/2000', periods=1000))
pdf = pd.DataFrame(np.random.randn(1000, 4), index=pser.index,
  columns=['A', 'B', 'C', 'D'])
psdf = ps.from_pandas(pdf)
psdf = psdf.cummax()
psdf.plot()

/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
